In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# Load data
directory = "/Users/varunursekar/Projects/advDeepLearning/bike_share/"

X_raw = np.load(directory+'X_top300_201701_201912_30_m.npy')

def window_average(x, fltr):
    window_shape = len(fltr)
    x = np.lib.stride_tricks.sliding_window_view(np.pad(x,(window_shape-1,0),constant_values=0),window_shape)
    return np.matmul(x,fltr)
    

def smooth_counts(X, filter_size=4, filter_type='even', var=1):
    
    if filter_type=='even':
        fltr = np.ones(filter_size)/filter_size
        
    elif filter_type=='gaussian':
        
        fltr = np.ones(filter_size)
        for i in range(filter_size):
            fltr[i] = np.exp(-i**2/var**2)
        fltr /= np.sum(fltr)
        fltr = fltr[::-1]
    
    print("Smoothing filter = ", fltr)
    
    X = np.apply_along_axis(lambda m: window_average(m, fltr), axis=1, arr=X)

    return X

def make_windows(X, window_length):
    
    num_samples = X.shape[1] - 2*window_length
    
    covariates = []
    labels = []
    
    for i in range(num_samples):
        
        covariates.append(X[:, i:i + window_length])
        labels.append(X[:, i + window_length:i + window_length + window_length])
    
    covariates = np.array(covariates)
    labels = np.array(labels)
    
    return covariates, labels

X_raw[:,:,:2] = smooth_counts(X_raw[:,:,:2], filter_size=3, filter_type='gaussian', var=1)
# X_raw[:,:,2] /= X_raw[:,:,2].max()
X_raw[:,:,2] = np.sin(2 * np.pi * X_raw[:,:,2] / (2 * 24))

X, y = make_windows(X_raw, window_length=10)

NUM_SAMPLES = len(X)
TRAIN_SAMPLES = int(0.5*NUM_SAMPLES)
VALID_SAMPLES = int(0.6*NUM_SAMPLES)
TEST_SAMPLES = int(0.8*NUM_SAMPLES)

X_train, y_train = X[:TRAIN_SAMPLES], y[:TRAIN_SAMPLES]
X_val, y_val = X[TRAIN_SAMPLES:VALID_SAMPLES], y[TRAIN_SAMPLES:VALID_SAMPLES]
X_test, y_test = X[VALID_SAMPLES:TEST_SAMPLES], y[VALID_SAMPLES:TEST_SAMPLES]

input_shape = tuple(list(X_train.shape)[1:])
output_shape = tuple(list(y_train.shape)[1:])
input_dims = np.product(input_shape)
output_dims = np.product(output_shape)

Smoothing filter =  [0.01321289 0.26538793 0.72139918]


In [3]:
input_shape

(300, 10, 3)

In [4]:
from models.fnn import FNN
from models.linear import LinearRegressor
from models.fclstm import FCLSTM
from models.stgcn import STGCN
from models.wavenet import GraphWaveNet

In [ ]:
default_params = {'num_nodes': 300,
                  'num_steps': 10,
                  'input_features': 3,
                  'output_features': 2,
                  'num_time_dims': 1,
                  'num_blocks': 2,
                  'block_units': [[64,16,64], [32,16,32]],
                  'block_kernels': [3,3],
                  'A': None,
                  'emb_dim': 32,
                  'time_modulation': False,
                  'time_mod_layers': 0,
                  'time_mod_activations':['relu','sigmoid'] ,
                  'l2_reg': 0.00}

model = STGCN(default_params)

In [5]:
model = GraphWaveNet({'num_nodes': 300,
                  'num_steps': 10,
                  'input_features': 3,
                  'output_features': 2,
                  'num_time_dims': 1,
                  'emb_dim': 32,
                  'num_blocks': 4,
                  'num_layers': 2,
                  'res_channels': 32,
                  'skip_channels': 32,
                  'dilation_channels': 32,
                  'end_channels': 32,
                  'time_modulation': False,
                  'time_mod_layers': 0,
                  'time_mod_activations': ['relu','sigmoid'],
                  'kernel_size': 2,
                      'use_gc': True
                  })

Metal device set to: Apple M1 Max


2022-04-23 16:35:39.639315: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-23 16:35:39.639549: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
model(X_train[:5])

<tf.Tensor: shape=(5, 300, 10, 2), dtype=float32, numpy=
array([[[[ 2.61002094e-01,  5.28316140e-01],
         [ 3.66829008e-01,  9.99626219e-01],
         [ 1.61089814e+00, -5.76052964e-02],
         ...,
         [ 6.23645186e-01,  2.75503248e-01],
         [-1.51429999e+00,  2.84769011e+00],
         [ 5.67196608e-01,  2.36710477e+00]],

        [[ 4.97668058e-01,  3.45410973e-01],
         [-3.65194947e-01,  2.66214281e-01],
         [ 1.40020752e+00,  4.81001347e-01],
         ...,
         [ 7.75381386e-01,  5.45698285e-01],
         [-8.59972835e-01,  1.93871999e+00],
         [ 9.49844956e-01,  1.48349226e+00]],

        [[-2.33319938e-01,  3.04164022e-01],
         [-6.43951178e-01,  6.14941001e-01],
         [ 1.18691432e+00,  2.96065956e-01],
         ...,
         [ 6.90723658e-01,  2.32593983e-01],
         [-5.25052607e-01,  1.68895459e+00],
         [ 7.21163675e-02,  9.58292603e-01]],

        ...,

        [[-7.03509897e-02,  6.39375687e-01],
         [-3.29200566e-01,

In [7]:
BATCH_SIZE = 64
MAX_EPOCHS = 15
    
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [8]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(1e-3))
print(model(X_train[:2]).shape)
model.summary()

(2, 300, 10, 2)
Model: "graph_wave_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  128       
                                                                 
 conv1d (Conv1D)             multiple                  1056      
                                                                 
 conv1d_1 (Conv1D)           multiple                  1056      
                                                                 
 conv1d_2 (Conv1D)           multiple                  1056      
                                                                 
 conv1d_3 (Conv1D)           multiple                  1056      
                                                                 
 conv1d_4 (Conv1D)           multiple                  1056      
                                                                 
 conv1d_5 (Conv1D)           multipl

In [9]:
history_model = model.fit(x=X_train, y=y_train[...,:-1], shuffle=True,
                    batch_size=BATCH_SIZE, epochs=MAX_EPOCHS,
                    validation_data=(X_val,y_val[...,:-1]),
                    callbacks=[callback_es])

Epoch 1/15


2022-04-23 16:35:51.388214: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2022-04-23 16:35:52.575558: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


416/416 [==============================] - ETA: 0s - loss: 3.8548

2022-04-23 16:37:21.845274: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


416/416 [==============================] - 98s 230ms/step - loss: 3.8548 - val_loss: 4.7293
Epoch 2/15
110/416 [======>.......................] - ETA: 1:02 - loss: 3.3016

KeyboardInterrupt: 